In [1]:
import os
import re

def next_who_filename(output_dir, prefix="WHO", ext="csv", width=3):
    """
    Ensures output_dir exists, then looks inside it for files like WHO_001.csv,
    WHO_002.csv, etc., finds the highest number, and returns the FULL PATH
    for the next file (e.g. './tables/WHO_003.csv').
    """
    # 1) make sure the directory exists
    os.makedirs(output_dir, exist_ok=True)

    # 2) build regex to match names like WHO_001.csv
    pattern = re.compile(rf"^{re.escape(prefix)}_(\d{{{width}}})\.{re.escape(ext)}$")
    max_num = 0

    # 3) scan the directory for existing matches
    for fname in os.listdir(output_dir):
        m = pattern.match(fname)
        if m:
            num = int(m.group(1))
            max_num = max(max_num, num)

    # 4) compute the next number, zero–padded
    next_num = max_num + 1
    new_fname = f"{prefix}_{next_num:0{width}d}.{ext}"

    # 5) return the *full* path, not just the basename
    return os.path.join(output_dir, new_fname), new_fname


In [20]:
from langdetect import detect, DetectorFactory
import pandas as pd
import os
import re

name = "hepatitis@2022-W23_dataset@Sheet1.tmp.xlsx"

# 1) point to your file
excel_path = f"../processed_tables/{name}"

# 2) load the 'data' sheet
df_data = pd.read_excel(excel_path, sheet_name='data')

# 3) extract headers and make comma-separated string
columns = df_data.columns.tolist()
columns_str = ','.join(columns)

# 4) for each column, find the first non-null value (or None if none exists)
first_values = []
for col in columns:
    non_null = df_data[col].dropna()
    if len(non_null) > 0:
        first_values.append(str(non_null.iloc[0]))
    else:
        first_values.append(None)

# 5) make a comma-separated string of those first values
#    (you’ll get 'None' only for columns that have no values at all)
values_str = ','.join(v if v is not None else 'None' for v in first_values)

output_folder = "../tables"  # change this to your real path
os.makedirs(output_folder, exist_ok=True)

# Compute next filename:
out_path, table_name = next_who_filename(output_folder)
# Save:
df_data.to_csv(out_path, index=False)

# read dictionary
df_dict = pd.read_excel(excel_path, sheet_name='dictionary')
df_dict.to_string()


prompt = "You are an schema matching ai agent. You will be provided with column names and column values as well as unified schema. Your task is to find columns that are matching with unifed schema. First I will provide to you column names and column values and unified schema. Then you will provided with dictionary. Output in the following format: {table_name},{column_id},{unified_label}. Note that table name will be provided to you, column id always starts with 0, unified label should be taken from a unified schema. Also, not all processed_tables have relative column names and you will be working in domain of epidimiological data."

output_txt = os.path.join("../output", "output.txt")

# Open in write mode ('w' will always recreate the file)
with open(output_txt, "w", encoding="utf-8") as f:
    f.write(f"Table name: {table_name}\n\n")
    f.write("Column names:\n")
    f.write(columns_str + "\n\n")
    f.write("Column values:\n")
    f.write(values_str + "\n\n")
    f.write("Dictionary: \n")
    f.write(df_dict.to_string() + "\n")
    f.write("\n")

print(f"Saved table {table_name} with all outputs from {excel_path}")


#DetectorFactory.seed = 0

#text_sample = ' '.join(str(value) for value in df_data.iloc[0].values)

#language = detect(text_sample)

#print(f"Detected language: {language}")
#print(f"Table name: {out_path}")
# 6) print out results
#print("Column names:")
#print(columns_str)
#print("")
#print("Column values:")
#print(values_str)


Saved table WHO_039.csv with all outputs from ../processed_tables/hepatitis@2022-W23_dataset@Sheet1.tmp.xlsx


In [17]:
df_dict = pd.read_excel(excel_path, sheet_name='dictionary')
print("Dictionary:")
print(df_dict)  # in Jupyter this will render as a nice table

Dictionary:
            variable_name                                variable_definition
0                RecordId                   Unique identifier for the record
1         AdenoRespResult                 Adenovirus respiratory test result
2            AdenoSeqDone   Indicator if adenovirus sequencing was performed
3        AdenoSerumResult                       Adenovirus serum test result
4        AdenoStoolResult                       Adenovirus stool test result
5           AdenotypeDone       Indicator if adenovirus typing was performed
6       AdenotypeSpecimen           Specimen type used for adenovirus typing
7              Adenovtype                         Adenovirus type identified
8   AdenoWholeBloodResult                 Adenovirus whole blood test result
9          AdmittedICUHDU                   Indicator if admitted to ICU/HDU
10      AdmittedTransUnit           Indicator if admitted to transplant unit
11            ADVSequence  If sequencing done, what adenovirus t

In [ ]:
#TODO: Append all output to one .txt file to put simply to chat GPT with a created prompt

In [1]:
LABEL_MAP = {
    # Date
    "Vaccination_date": "date",
    "Date_report":"date",
    "Date_onset":  "date",
    "Date_confirmation": "date",
    "Date_of_first_consultation":"date",
    "Date_hospitalisation":  "date",
    "Date_discharge_hospital": "date",
    "Date_admission_ICU":   "date",
    "Date_discharge_ICU":  "date",
    "Date_isolation":  "date",
    "Date_death":  "date",
    "Date_recovered":  "date",
    "Travel_history_entry": "date",
    "Travel_history_start":  "date",
    "Date_entry":  "date",
    "Date_last_modified": "date",

    # ID
    "Contact_ID": "id",
    "ID": "id",

    #Gender
    "Gender": "gender",
    "Sex_at_birth": "gender",
    "Gender_other": "gender",
    "Sex_at_birth_other": "gender",

    #Location
    "Travel_history_location": "location",
    "Location_information": "location",

    # Contact setting
    "Contact_setting": "contact_setting",
    "Contact_setting_other": "contact_setting",

    # demographic
    "Race": "demographic",
    "Ehtnicity": "demographic",

    # Medical Boolean
    "Healthcare_worker": "medical_boolean",
    "Previous_infection": "medical_boolean",
    "Pregnancy_Status": "medical_boolean",
    "Vaccination":  "medical_boolean",
    "Hospitalised":  "medical_boolean",
    "Intensive_care":  "medical_boolean",
    "Home_monitoring":  "medical_boolean",
    "Isolated": "medical_boolean",
    "Contact_with_case": "medical_boolean",
    "Travel_history": "medical_boolean",

    # Sourec
    "Source": "source",
    "Source_II": "source",
    "Source_III": "source",
    "Source_IV": "source",
}